#  Number Analysis
This notebook calculates all of the important numbers that are reported in the manuscript (e.g. JJA SIA Anomaly in 2023 from NSIDC and CESM2-NUDGE)
- JJA SIA Anomaly JJA 2023 NSIDC
- JJA SIA Raw Value JJA 2023 NSIDC
- JJA SIA Climatology JJA 2023 NSIDC

- JJA SIA Anomaly JJA 2023 CESM2-NUDGE
- JJA SIA Anomaly JJA 2023 CESM2-NUDGE Regional Anomalies
- JJA SIA Anomaly JJA 2023 Persistence
- JJA SIA Anomaly JJA 2023 Persistence Regional Anomalies
- JJA SIA Anomaly JJA 2023 Persistence/CESM2-NUDGE
- JJA SIA Anomaly JJA 2023 CESM2-NO-ENSO-NUDGE

In [2]:
import numpy as np
import xarray as xr

/glade/work/zespinosa/conda-envs/cenv/lib/python3.8/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.27.0
  warnings.warn(


# Load Data

In [3]:
# Load Nudge Data
si_nsidc = xr.open_dataset("/glade/work/zespinosa/Projects/SI-Antarctic/data/si_NSIDC_SH_anoms.nc")
si_cesm2 = xr.open_dataset("/glade/work/zespinosa/Projects/SI-Antarctic/data/si_CESM_SH_si-anoms.nc")

si_nsidc_raw = xr.open_dataset("/glade/work/zespinosa/Projects/SI-Antarctic/data/si_NSIDC_SH.nc")
si_cesm2_raw = xr.open_dataset("/glade/work/zespinosa/Projects/SI-Antarctic/data/si_CESM_SH_si.nc")

si_regions_nsidc = xr.open_dataset("/glade/work/zespinosa/Projects/SI-Antarctic/data/si_regions_NSIDC_anoms.nc")
si_regions_cesm2 = xr.open_dataset("/glade/work/zespinosa/Projects/SI-Antarctic/data/si_regions_CESM_anoms.nc")

ERROR 1: PROJ: proj_create_from_database: Open of /glade/work/zespinosa/conda-envs/cenv/share/proj failed


In [5]:
# Load Ensemble Data
ENSEMBLE_MEMBERS = ["1980", "1985", "1989", "1990", "1993", "1994", "1998", "1999", "2000", "2003", "2004", "2005", "2006", "2007", "2009", "2012", "2013", "2014", "2018", "2020", "2021"]

def load_ensemble(file_name, ens_mems, drop_vars=[], include_year=False):
    """
    Load the processed data files for each ensemble member and return a concatenated dataset.
    """
    ens = []
    for mem in ens_mems: 
        try:
            if include_year:
                path = f"../data/persistence_ensemble/{mem}/{mem}{file_name}"
            else:
                path = f"../data/persistence_ensemble/{mem}/{file_name}"

            cesm2_ens = xr.open_dataset(
                path,
                chunks="auto", 
                drop_variables=drop_vars
            )
            ens.append(cesm2_ens)
        except Exception as e:
            print(f"Could not load {mem}")
            print(e)

    ens = xr.concat(ens, "year")

    return ens

si_cesm2_ens = load_ensemble(file_name=f"si_CESM_SH_anoms.nc", ens_mems=ENSEMBLE_MEMBERS).mean("year")
si_regions_cesm2_ens = load_ensemble(file_name=f"si_regions_CESM_anoms.nc", ens_mems=ENSEMBLE_MEMBERS).mean("year")


In [6]:
# Load ENSO Data
si_cesm2_enso = xr.open_dataset("/glade/work/zespinosa/Projects/SI-Antarctic/data/enso_si_CESM_SH_anoms.nc")
si_regions_cesm2_enso = xr.open_dataset("/glade/work/zespinosa/Projects/SI-Antarctic/data/enso_si_regions_CESM_anoms.nc")

# Print Numbers

In [7]:
START_CLIM = "1980-01-01"
END_CLIM = "2020-01-01"

def round(x):
    """Round the input to 2 decimal places"""
    return np.around(x, 2) 

def season_mean(ds, calendar="standard"):
    """ https://docs.xarray.dev/en/stable/examples/monthly-means.html """
    # Make a DataArray with the number of days in each month, size = len(time)
    month_length = ds.time.dt.days_in_month

    # Calculate the weights by grouping by 'time.season'
    weights = (
        month_length.groupby("time.season") / month_length.groupby("time.season").sum()
    )

    # Test that the sum of the weights for each season is 1.0
    np.testing.assert_allclose(weights.groupby("time.season").sum().values, np.ones(4))

    # Calculate the weighted average
    return (ds * weights).groupby("time.season").sum(dim="time")

In [10]:
SI_METRIC = 'sia' # 'sia'
SI_METRIC_LABEL = 'Sea Ice Area' #'Sea Ice Area'
SEASON = 'QS-DEC'
SEASON_TIME = "2023-06"
REF_PERIOD = ["1980-01-01", "2020-01-01"]

In [11]:
# Climatologies
si_nsidc_clim = season_mean(si_nsidc_raw[SI_METRIC].sel(time=slice(REF_PERIOD[0], REF_PERIOD[1])), calendar="standard")
si_cesm2_clim = season_mean(si_cesm2_raw[SI_METRIC].sel(time=slice(REF_PERIOD[0], REF_PERIOD[1])), calendar="standard")

# Print JJA Climatologies
print("NSIDC Climatology: ", round(si_nsidc_clim.sel(season="JJA").values))
print("CESM2 Climatology: ", round(si_cesm2_clim.sel(season="JJA").values))

NSIDC Climatology:  13.71
CESM2 Climatology:  8.21


In [12]:
nsidc_std = si_nsidc[SI_METRIC].resample(time=SEASON).mean(dim="time").sel(time=slice("1979-01", "2023-08")).std('time')

In [23]:
# Find previous largest seasonal anomalies
si_nsidc_anoms = si_nsidc[SI_METRIC].resample(time=SEASON).mean(dim="time")
si_nsidc_anoms.sortby(si_nsidc_anoms)

<xarray.DataArray 'sia' (time: 180)>
array([-2.22052291, -1.65916501, -1.29925375, -1.2471531 , -1.20523532,
       -1.18899733, -1.144972  , -1.07233387, -1.04345284, -0.92368117,
       -0.89112192, -0.84239738, -0.80122173, -0.79505022, -0.78644132,
       -0.73818616, -0.72727334, -0.7209667 , -0.69358766, -0.68873233,
       -0.68004428, -0.67502603, -0.67445808, -0.6243521 , -0.62266685,
       -0.61185568, -0.60569034, -0.54457268, -0.53436932, -0.5156794 ,
       -0.50322806, -0.50174321, -0.42683965, -0.41825717, -0.40834953,
       -0.40527599, -0.39956676, -0.39673854, -0.39033702, -0.36534899,
       -0.36461293, -0.36433716, -0.33429474, -0.33195897, -0.29077812,
       -0.28934103, -0.28759332, -0.28082964, -0.26036571, -0.25903702,
       -0.25831464, -0.25246514, -0.25197528, -0.24065838, -0.23996659,
       -0.23722617, -0.23279835, -0.23102166, -0.22890709, -0.22374845,
       -0.21852777, -0.21664796, -0.21265712, -0.1961249 , -0.19319993,
       -0.18612233, -0.17612261, -0.17382346, -0.17278364, -0.17133969,
       -0.17131763, -0.15700116, -0.15455426, -0.15147468, -0.14141242,
       -0.14067703, -0.13718676, -0.13588151, -0.12339724, -0.12329521,
       -0.11799339, -0.11740855, -0.11661029, -0.11617638, -0.11572733,
       -0.10821306, -0.09147084, -0.0908286 , -0.07767688, -0.07681446,
       -0.06478547, -0.05912791, -0.05278427, -0.05217845, -0.05001626,
       -0.04948932, -0.04020672, -0.03424892, -0.03414386, -0.0337721 ,
       -0.02669076,  0.01811455,  0.02804584,  0.03739378,  0.05836095,
        0.05922476,  0.06189764,  0.09163779,  0.11644849,  0.11835913,
        0.12079506,  0.13211704,  0.14046494,  0.14488727,  0.14679384,
        0.17268645,  0.1779723 ,  0.18172169,  0.18566916,  0.19264205,
        0.19628694,  0.19711883,  0.19999081,  0.20359557,  0.21720524,
        0.22106565,  0.23267338,  0.23535223,  0.23646808,  0.24146098,
        0.24707852,  0.24834608,  0.25474889,  0.26156232,  0.26904219,
        0.28737114,  0.30694985,  0.30726948,  0.30791748,  0.31815591,
        0.33171574,  0.33274815,  0.33345977,  0.33424179,  0.3352397 ,
        0.34959734,  0.36151048,  0.36941982,  0.37100017,  0.39353169,
        0.39465532,  0.3970061 ,  0.4136575 ,  0.4163283 ,  0.41635219,
        0.41816347,  0.42892135,  0.43143901,  0.43346527,  0.43463025,
        0.4452473 ,  0.4498401 ,  0.48115396,  0.50717751,  0.50791592,
        0.53117737,  0.70029435,  0.70468415,  0.78021682,  0.82168979,
        0.83935368,  0.85756195,  0.98253975,  1.00719121,  1.01773448,
        1.04841257,  1.09173768,  1.10372583,  1.16980519,  1.18280677])
Coordinates:
  * time     (time) datetime64[ns] 2023-06-01 2023-09-01 ... 2015-03-01

In [14]:
print(f'NSIDC JJA 2023 {SI_METRIC} anomaly: ', round(si_nsidc[SI_METRIC].resample(time=SEASON).mean(dim="time").sel(time=SEASON_TIME).values)[0])
print(f'NSIDC JJA 2023 {SI_METRIC} std anomaly: ', round(si_nsidc[SI_METRIC].resample(time=SEASON).mean(dim="time").sel(time=SEASON_TIME).values)[0] / nsidc_std)
print(f'CESM2-NUDGE JJA 2023 {SI_METRIC} anomaly: ', round(si_cesm2[SI_METRIC].resample(time=SEASON).mean(dim="time").sel(time='2023-06').values)[0])
print(f'CESM2-NUDGE JJA 2024 {SI_METRIC} anomaly: ', round(si_cesm2[SI_METRIC].resample(time=SEASON).mean(dim="time").sel(time='2024-06').values)[0])

print(f'NO ENSO CESM2-NUDGE JJA 2023 {SI_METRIC} anomaly: ', round(si_cesm2_enso[SI_METRIC].resample(time=SEASON).mean(dim="time").sel(time=SEASON_TIME).values)[0])

nsidc_jja_23_raw = round(si_nsidc_raw[SI_METRIC].resample(time=SEASON).mean(dim="time").sel(time=SEASON_TIME).values)[0]
cesm2_jja_23_raw = round(si_cesm2_raw[SI_METRIC].resample(time=SEASON).mean(dim="time").sel(time=SEASON_TIME).values)[0]
print(f'NSIDC JJA 2023 {SI_METRIC}: ', nsidc_jja_23_raw)
print(f'CESM2-NUDGE JJA 2023 {SI_METRIC}: ', cesm2_jja_23_raw)

regions = round(si_regions_nsidc[SI_METRIC].resample(time=SEASON).mean(dim="time").sel(time=SEASON_TIME))
print(f'NSIDC JJA 2023 {SI_METRIC} regions: ', [(r, regions.sel(region=r).values[0]) for r in regions.region.values])

regions = round(si_regions_cesm2[SI_METRIC].resample(time=SEASON).mean(dim="time").sel(time=SEASON_TIME))
print(f'CESM2-NUDGE JJA 2023 {SI_METRIC} regions: ', [(r, regions.sel(region=r).values[0]) for r in regions.region.values])

regions = round(si_regions_cesm2_enso[SI_METRIC].resample(time=SEASON).mean(dim="time").sel(time=SEASON_TIME))
print(f'NO_ENSO CESM2-NUDGE JJA 2023 {SI_METRIC} regions: ', [(r, regions.sel(region=r).values[0]) for r in regions.region.values])

print('NSIDC JJA 2023 DIFF: ', round(nsidc_jja_23_raw - si_nsidc_clim.sel(season="JJA").values))
print('CESM2 JJA 2023 DIFF: ', round(cesm2_jja_23_raw - si_cesm2_clim.sel(season="JJA").values))

print('NSIDC JJA 2023 %: ', round(1 - nsidc_jja_23_raw/si_nsidc_clim.sel(season="JJA").values))
print('CESM2 JJA 2023 %: ', round(1 - cesm2_jja_23_raw /si_cesm2_clim.sel(season="JJA").values))

NSIDC JJA 2023 sia anomaly:  -2.22
NSIDC JJA 2023 sia std anomaly:  <xarray.DataArray 'sia' ()>
array(-4.28835615)
CESM2-NUDGE JJA 2023 sia anomaly:  -2.45
CESM2-NUDGE JJA 2024 sia anomaly:  -1.81
NO ENSO CESM2-NUDGE JJA 2023 sia anomaly:  -2.39
NSIDC JJA 2023 sia:  11.47
CESM2-NUDGE JJA 2023 sia:  5.75
NSIDC JJA 2023 sia regions:  [('ross', -0.7), ('amundsen', 0.31), ('bellingshausen', -0.11), ('weddell', -0.74), ('south_indian', -0.73), ('south_west_pacific', -0.26)]
CESM2-NUDGE JJA 2023 sia regions:  [('ross', -0.96), ('amundsen', 0.09), ('bellingshausen', 0.02), ('weddell', -0.82), ('south_indian', -0.34), ('south_west_pacific', -0.45)]
NO_ENSO CESM2-NUDGE JJA 2023 sia regions:  [('ross', -0.95), ('amundsen', 0.11), ('bellingshausen', -0.01), ('weddell', -0.78), ('south_indian', -0.33), ('south_west_pacific', -0.44)]
NSIDC JJA 2023 DIFF:  -2.24
CESM2 JJA 2023 DIFF:  -2.46
NSIDC JJA 2023 %:  0.16
CESM2 JJA 2023 %:  0.3
